In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
class CFG:
    
    seed = 42
    debug = True  
    apex = False
    print_freq = 100
    num_workers = 2
    KAGGLE_NOTEBOOK = False
    
    if KAGGLE_NOTEBOOK:
        OUTPUT_DIR = '/kaggle/working/'
        train_datadir = '/kaggle/input/birdclef-2025/train_audio'
        train_csv = '/kaggle/input/birdclef-2025/train.csv'
        test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
        submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
        taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
        spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
    else:
        OUTPUT_DIR = '../data/result/'
        train_datadir = '../data/raw/train_audio/'
        train_csv = '../data/raw/train.csv'
        test_soundscapes = '../data/raw/test_soundscapes/'
        submission_csv = '../data/raw/sample_submission.csv'
        taxonomy_csv = '../data/raw/taxonomy.csv'
        spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
        MODEL_DIR = "../models/"


cfg = CFG()

In [6]:
train_label = pd.read_csv(cfg.train_csv)
submission = pd.read_csv(cfg.submission_csv)
taxonomy = pd.read_csv(cfg.taxonomy_csv)

In [19]:
train_label.head()

# secoundary labels存在
train_label[train_label["secondary_labels"] != "['']"]

train_label[train_label["type"] != "['']"]

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
9,126247,"['65448', '22976', '476538']",['advertisement call'],126247/XC941297.ogg,XC,3.5,https://xeno-canto.org/941297,9.0465,-79.3024,Leptodactylus insularum,Spotted Foam-nest Frog,Chris Harrison,cc-by-nc-sa 4.0
20,134933,[''],['advertisement call'],134933/XC941298.ogg,XC,4.0,https://xeno-canto.org/941298,8.6260,-80.1392,Espadarana prosoblepon,Emerald Glass Frog,Chris Harrison,cc-by-nc-sa 4.0
130,22333,[''],['advertisement call'],22333/XC890507.ogg,XC,5.0,https://xeno-canto.org/890507,12.1205,-68.9658,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Chris Harrison,cc-by-nc-sa 4.0
131,22333,[''],['territorial call'],22333/XC894982.ogg,XC,4.0,https://xeno-canto.org/894982,12.1187,-68.2912,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Baudewijn Odé,cc-by-nc-sa 4.0
177,22973,[''],['flight call'],22973/XC167037.ogg,XC,4.0,https://xeno-canto.org/167037,3.8284,-67.9020,Leptodactylus fuscus,Whistling Grass Frog,Ottavio Janni,cc-by-nc-sa 3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28533,ywcpar,[''],['uncertain'],ywcpar/XC893653.ogg,XC,5.0,https://xeno-canto.org/893653,5.3395,-72.3831,Amazona ochrocephala,Yellow-crowned Parrot,Nerieth Leuro,cc-by-nc-sa 4.0
28534,ywcpar,[''],['flight call'],ywcpar/XC939157.ogg,XC,2.0,https://xeno-canto.org/939157,3.6867,-61.0326,Amazona ochrocephala,Yellow-crowned Parrot,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0
28535,ywcpar,[''],['call'],ywcpar/XC939934.ogg,XC,5.0,https://xeno-canto.org/939934,1.7914,-61.1375,Amazona ochrocephala,Yellow-crowned Parrot,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0
28537,ywcpar,[''],['call'],ywcpar/XC946300.ogg,XC,4.0,https://xeno-canto.org/946300,3.5026,-76.3552,Amazona ochrocephala,Yellow-crowned Parrot,Cristian Rodas,cc-by-nc-sa 4.0


In [10]:
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,soundscape_8358733_5,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
1,soundscape_8358733_10,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
2,soundscape_8358733_15,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854


In [ ]:
# ラベルと生物名の対応
taxonomy.head()

,primary_label,inat_taxon_id,scientific_name,common_name,class_name
0,1139490,1139490,Ragoniella pulchella,Ragoniella pulchella,Insecta
1,1192948,1192948,Oxyprora surinamensis,Oxyprora surinamensis,Insecta
2,1194042,1194042,Copiphora colombiae,Copiphora colombiae,Insecta
3,126247,126247,Leptodactylus insularum,Spotted Foam-nest Frog,Amphibia
4,1346504,1346504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Insecta
